<a href="https://colab.research.google.com/github/madham32/Recommender_System/blob/master/ML_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tensorflow-gpu

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
from google.colab import files
files.upload()  #this will prompt you to upload the kaggle.json

In [0]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json  # set permission

kaggle.json


In [0]:
!kaggle datasets download -d netflix-inc/netflix-prize-data -p /content/gdrive/My\ Drive/kaggle/netflix

In [0]:
import os
os.chdir('gdrive/My Drive/kaggle/netflix')  #change dir

# !unzip netflix-prize-data.zip


In [0]:
import csv
import pandas as pd
from keras.models import Sequential
from keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate, Activation, Reshape

Using TensorFlow backend.


In [0]:
i=0
probe_file1=open('combined_data_1.txt','r') 
probe_file2=open('combined_data_2.txt','r') 
probe_file3=open('combined_data_3.txt','r') 
probe_file4=open('combined_data_4.txt','r') 

new_file=open('NEW_probe_data.txt','w')

def add_data(probefile, newfile, i):
    
    for line in probefile:
        if ':' in line:
            i+=1
            continue
        line=line.strip()+','+str(i)+'\n'
        newfile.write(line) 
    return i
    

i=add_data(probe_file1, new_file, i)
print(i)
i=add_data(probe_file2, new_file, i)
print(i)
i=add_data(probe_file3, new_file, i)
print(i)
i=add_data(probe_file4, new_file, i)
print(i)

4499
9210
13367
17770


In [0]:
data = pd.read_csv( "NEW_probe_data.txt", header=0, 
 delimiter=",", quoting=3,names=['user_id','rating','timestamp','movie_id'] )

In [0]:
data

In [0]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.2, random_state=42)

In [0]:
n_users = len(data.user_id.unique())

n_movies = len(data.movie_id.unique())


In [0]:
# creating book embedding path
movie_input = Input(shape=[1], name="Movie-Input")
movie_embedding = Embedding(n_movies+1, 4, name="Movie-Embedding")(movie_input)
movie_vec = Flatten(name="Flatten-Movies")(movie_embedding)

# creating user embedding path
user_input = Input(shape=[1], name="User-Input")
user_embedding = Embedding(n_users+1, 4, name="User-Embedding")(user_input)
user_vec = Flatten(name="Flatten-Users")(user_embedding)

# performing dot product and creating model
prod = Dot(name="Dot-Product", axes=1)([movie_vec, user_vec])
model = Model([user_input, movie_input], prod)
model.compile('adam', 'mean_squared_error')

In [0]:
from keras.models import Model


# creating book embedding path
movie_input = Input(shape=[1], name="Movie-Input")
movie_embedding = Embedding(n_movies+1, 4, name="Movie-Embedding")(movie_input)
movie_vec = Flatten(name="Flatten-Movies")(movie_embedding)

# creating user embedding path
user_input = Input(shape=[1], name="User-Input")
user_embedding = Embedding(n_users+1, 4, name="User-Embedding")(user_input)
user_vec = Flatten(name="Flatten-Users")(user_embedding)

# concatenate features
conc = Concatenate()([movie_vec, user_vec])

# add fully-connected-layers
fc1 = Dense(128, activation='relu')(conc)
fc2 = Dense(32, activation='relu')(fc1)
out = Dense(1)(fc2)

# Create model and compile it
model2 = Model([user_input, movie_input], out)
model2.compile('adam', 'mean_squared_error')

In [0]:

from keras.models import load_model

if os.path.exists('regression_model2.h5'):
    model2 = load_model('regression_model2.h5')
else:
    history = model2.fit([train.user_id, train.movie_id], train.rating, epochs=1, verbose=1)
    model2.save('regression_model2.h5')


In [0]:
history.history['loss']

[1.0011890296106545]

In [0]:
import numpy as np


# Creating dataset for making recommendations for the first user
movie_data = np.array(list(set(data.movie_id)))
movie_data[:5]


user = np.array([1 for i in range(len(movie_data))])
user[:5]

In [0]:

predictions = model2.predict([user, movie_data])

predictions = np.array([a[0] for a in predictions])

recommended_movie_ids = (-predictions).argsort()[:5]

recommended_movie_ids

array([ 8075,  5658,  3455,  7229, 14960])

In [0]:
predictions[recommended_movie_ids]

array([15.407293 , 14.199906 ,  4.7365074,  4.735477 ,  4.7207274],
      dtype=float32)

In [0]:
model2.evaluate([test.user_id, test.movie_id], test.rating)

20095955/20095955 [==============================] - 801s 40us/step


0.9935715879386653